<a href="https://colab.research.google.com/github/muyuuuu/colab/blob/main/adversarial-denoise/adp(ICML%20202021).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

import numpy as np

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.49, 0.48, 0.44), (0.24, 0.24, 0.26))])

batch_size = 32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
class Score(nn.Module):
    def __init__(self):
        super().__init__()

        nef = 32
        self.u_net = nn.Sequential(
            # input is (nc) x 28 x 28
            nn.Conv2d(3, nef, 16, stride=2, padding=2),
            # nn.Softplus(),
            nn.GroupNorm(4, nef),
            nn.ELU(),
            # state size. (nef) x 14 x 14
            nn.Conv2d(nef, nef * 2, 4, stride=2, padding=2),
            nn.GroupNorm(4, nef * 2),
            # nn.Softplus(),
            nn.ELU(),
            # state size. (nef*2) x 7 x 7
            nn.Conv2d(nef * 2, nef * 4, 5, stride=1, padding=2),
            nn.GroupNorm(4, nef * 4),
            # nn.Softplus(),
            nn.ELU(),
            # state size. (nef*4) x 3 x 3
            nn.ConvTranspose2d(nef * 4, nef * 2, 5, stride=1, padding=1),
            nn.GroupNorm(4, nef * 2),
            # nn.Softplus(),
            nn.ELU(),
            # state size. (nef*2) x 7 x 7
            nn.ConvTranspose2d(nef * 2, nef, 4, stride=2, padding=1),
            nn.GroupNorm(4, nef),
            # nn.Softplus(),
            nn.ELU(),
            # state size. (nef) x 14 x 14
            nn.ConvTranspose2d(nef, 3, 4, stride=2, padding=1),
            # nn.Softplus()

            nn.ELU()
        )
        self.fc = nn.Sequential(
            nn.Linear(3 * 32 * 32, 1024),
            nn.LayerNorm(1024),
            nn.ELU(),
            nn.Linear(1024, 3 * 32 * 32)
        )

    def forward(self, x):

        score = self.u_net(x)
        score = self.fc(score.view(x.shape[0], -1)).view(
            x.shape[0], 3, 32, 32)

        return score

In [5]:
net = Score().to(device)
net.load_state_dict(torch.load('score.pth'))
net.eval()

Score(
  (u_net): Sequential(
    (0): Conv2d(3, 32, kernel_size=(16, 16), stride=(2, 2), padding=(2, 2))
    (1): GroupNorm(4, 32, eps=1e-05, affine=True)
    (2): ELU(alpha=1.0)
    (3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
    (4): GroupNorm(4, 64, eps=1e-05, affine=True)
    (5): ELU(alpha=1.0)
    (6): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (7): GroupNorm(4, 128, eps=1e-05, affine=True)
    (8): ELU(alpha=1.0)
    (9): ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (10): GroupNorm(4, 64, eps=1e-05, affine=True)
    (11): ELU(alpha=1.0)
    (12): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (13): GroupNorm(4, 32, eps=1e-05, affine=True)
    (14): ELU(alpha=1.0)
    (15): ConvTranspose2d(32, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (16): ELU(alpha=1.0)
  )
  (fc): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=True)
 

In [25]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()
model.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [1]:
delta = 1e-5
lambda_ = 0.05
tao = 0.001
alpha = None
correct = 0
total = 0
sigma = 0.1


with torch.no_grad():
    for i, data in enumerate(testloader, 0):

        inputs, labels = data
        predicted = torch.zeros(10, inputs.size()[0])

        for i in range (10):

            inputs = inputs.to(device)
            inputs += torch.randn_like(inputs) * sigma
            torch.clamp(inputs, 0.0, 1.0)

            for _ in range (10):

                denoise = net(inputs)

                inputs_ = inputs + delta * denoise

                f1 = torch.bmm(inputs.view(inputs.size()[0], 1, -1), inputs_.view(inputs_.size()[0], -1, 1))
                f2 = torch.bmm(inputs.view(inputs.size()[0], 1, -1), inputs.view(inputs_.size()[0], -1, 1))
                f = torch.div(f1, f2)

                alpha = torch.squeeze(torch.clamp(lambda_ * delta / (1.0 - f), min=0.00001, max=0.1))
                inputs = torch.clamp(inputs.detach() + alpha * denoise, 0.0, 1.0)

                if (torch.norm(net(inputs), p=1) < tao):
                    break

            outputs = model(inputs)
            _, idx = torch.max(outputs.data, 1)
            predicted[i] = idx

        predicted, _ = torch.mode(predicted, 0)
        # break
        print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # break
    
    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

NameError: ignored